In [1]:
# https://blog.naver.com/ostin1038/222113905187

In [1]:
import pandas as pd
import numpy as np

In [2]:
D = pd.read_csv('data/lung_cancer.csv')
D.head()

,Member_ID,Age,Smokes,Alkhol,familyhistory,Result
0,91550,35,3,4,0,1
1,915664,27,20,5,0,1
2,915691,30,0,2,0,0
3,915940,28,0,1,1,0
4,91594602,68,4,6,0,1


In [3]:
D['gen'] = D.Age.apply(lambda x : x < 20 and '소년' or x < 40 and '청년' or x < 60 and '중년' or '노년' )
D.gen

0       청년
1       청년
2       청년
3       청년
4       노년
        ..
1293    청년
1294    노년
1295    노년
1296    중년
1297    중년
Name: gen, Length: 1298, dtype: object

In [4]:
D[D.gen=='청년'].Age.describe()

count    616.000000
mean      30.178571
std        5.449302
min       21.000000
25%       26.000000
50%       30.500000
75%       34.250000
max       39.000000
Name: Age, dtype: float64

In [8]:
# Q1. 중년 연령대의 폐암 환자 비중을 확인하고 음주량 평균과 중앙값을 구하시오 
# (소수 두번째짜리까지 기록, 셋째짜리에서 버림)

In [7]:
cancer = D[(D.gen == '중년')&(D.Result== 1)].shape[0]
total = D[(D.gen == '중년')].shape[0]
cancer, total, cancer/total

(264, 396, 0.6666666666666666)

In [9]:
D[D.gen == '중년']['Alkhol'].mean()

3.5

In [10]:
D[D.gen == '중년']['Alkhol'].median()

3.5

In [12]:
# Q2
"""
흡연과 폐암이 서로 연관이 있는지 통계적으로 확인하고자 한다. 
먼저 흡연량 21 이상이면 '고' 11~20은 '중', 10 이하는 '소'로 흡연자를 
3가지 등급으로 구분한 후 카이스퀘어검정으로 흡연과 폐암의 연관성을 확인한다. 
흡연 등급자와 폐암 유무에 따른 카이스퀘어 검정 검정통계량을 
소수점 2자리까지구하시오(셋째자리에서 버림)
"""

"\n흡연과 폐암이 서로 연관이 있는지 통계적으로 확인하고자 한다. \n먼저 흡연량 21 이상이면 '고' 11~20은 '중', 10 이하는 '소'로 흡연자를 \n3가지 등급으로 구분한 후 카이스퀘어검정으로 흡연과 폐암의 연관성을 확인한다. \n흡연 등급자와 폐암 유무에 따른 카이스퀘어 검정 검정통계량을 \n소수점 2자리까지구하시오(셋째자리에서 버림)\n"

In [11]:
D['smokelvl'] = D.Smokes.apply(lambda x : x >= 21 and '고' or x >= 11 and '중' or '소' )

In [15]:
D[['Smokes', 'smokelvl']].groupby(by='smokelvl').describe().T

smokelvl               고           소           중
Smokes count  176.000000  374.000000  748.000000
       mean    27.000000    5.058824   17.264706
       std      3.684717    3.738326    3.195041
       min     22.000000    0.000000   12.000000
       25%     25.000000    2.000000   15.000000
       50%     25.000000    4.000000   20.000000
       75%     30.000000   10.000000   20.000000
       max     34.000000   10.000000   20.000000

In [16]:
D[['Result', 'smokelvl']]

,Result,smokelvl
0,1,소
1,1,중
2,0,소
3,0,소
4,1,소
...,...,...
1293,0,중
1294,1,중
1295,1,중
1296,1,고


In [21]:
D[['Result', 'smokelvl']].pivot_table(index='smokelvl',columns='Result', aggfunc="count")

smokelvl
고
소
중


In [25]:
D['cancer'] = D['Result'].astype(str)

In [27]:
D[['Result', 'smokelvl','cancer']].pivot_table(index='smokelvl',columns='cancer', aggfunc="count")

Result     
cancer        0    1
smokelvl            
고            44  132
소           264  110
중           374  374

In [28]:
D2 = D[['Result', 'smokelvl']].copy()
D2['cancerlvl'] = D2.Result.apply(lambda x : x == 1 and '폐암' or '정상')

In [29]:
D2.head()

,Result,smokelvl,cancerlvl
0,1,소,폐암
1,1,중,폐암
2,0,소,정상
3,0,소,정상
4,1,소,폐암


In [30]:
D2.pivot_table(index='smokelvl', columns='cancerlvl', aggfunc="count")

Result     
cancerlvl     정상   폐암
smokelvl             
고             44  132
소            264  110
중            374  374

In [31]:
D2P = D2.pivot_table(index='smokelvl', columns='cancerlvl', aggfunc="count")

In [33]:
import scipy.stats as stats

In [35]:
chi2, p, dof, expected = stats.chi2_contingency(observed=D2P.values) # 통계량, pvalue, 자유도, 기댓값
chi2, p, dof, expected

(104.32556248305772,
 2.2181552840800698e-23,
 2,
 array([[ 92.47457627,  83.52542373],
        [196.50847458, 177.49152542],
        [393.01694915, 354.98305085]]))

In [36]:
stats.chi2_contingency(observed=D2P.T.values) # 전치 또한 카이제곱 검정 결과는 같다.

(104.32556248305774,
 2.218155284080054e-23,
 2,
 array([[ 92.47457627, 196.50847458, 393.01694915],
        [ 83.52542373, 177.49152542, 354.98305085]]))

In [38]:
#
_ = """
Q3. 로지스틱 회귀분석을 통해 폐암 여부를 예측하는 단순 분류 모델을 만들고자 한다.
먼저 테스트셋 트레인셋을 분할하는 작업으로 Member_ID 끝자리가 4의 배수로 끝나는 경우를 테스트셋, 
나머지는 테스트셋으로 분할한다. 트레인셋을 바탕으로 로지스틱 회귀분석 모델을 만
( 판다스 get_dummies 함수 활용, 연령대는 청년, 흡연량은 중에 해당하는 변수 제거한다 ) 
연령대(더미변수), 음주량, 가족력을 독립변수로 폐암여부들되 연령대와 폐암 등급은 더미변수로 변환한다
를 종속변수로 예측 모델을 만들어 수행하여 
폐암인 1의 결과값에 대한 F-1 Score를 확인하라 

이때 폐암으로 예측될 확률이 0.9 이상인 경우 폐암으로 예측하는 조건을 추가
( 옵션 : seed:13, Penalty: I2, solver:newton-cg) 
"""

In [39]:
D3 = D.copy()

In [40]:
D3.head()

,Member_ID,Age,Smokes,Alkhol,familyhistory,Result,gen,smokelvl,cencer,cancer
0,91550,35,3,4,0,1,청년,소,1,1
1,915664,27,20,5,0,1,청년,중,1,1
2,915691,30,0,2,0,0,청년,소,0,0
3,915940,28,0,1,1,0,청년,소,0,0
4,91594602,68,4,6,0,1,노년,소,1,1


In [41]:
D3_1 = pd.concat([D3,pd.get_dummies(D3.smokelvl),pd.get_dummies(D3.gen)], axis=1)
D3_1.head()

,Member_ID,Age,Smokes,Alkhol,familyhistory,Result,gen,smokelvl,cencer,cancer,고,소,중,노년,소년,중년,청년
0,91550,35,3,4,0,1,청년,소,1,1,0,1,0,0,0,0,1
1,915664,27,20,5,0,1,청년,중,1,1,0,0,1,0,0,0,1
2,915691,30,0,2,0,0,청년,소,0,0,0,1,0,0,0,0,1
3,915940,28,0,1,1,0,청년,소,0,0,0,1,0,0,0,0,1
4,91594602,68,4,6,0,1,노년,소,1,1,0,1,0,1,0,0,0


In [42]:
D3_2 = D3_1.drop(labels=['중','청년'], axis=1).copy()

In [44]:
D3_2['mode'] =  np.where(D3.Member_ID % 4 == 0, "TEST", "TRAIN")
D3_2.head()

,Member_ID,Age,Smokes,Alkhol,familyhistory,Result,gen,smokelvl,cencer,cancer,고,소,노년,소년,중년,mode
0,91550,35,3,4,0,1,청년,소,1,1,0,1,0,0,0,TRAIN
1,915664,27,20,5,0,1,청년,중,1,1,0,0,0,0,0,TEST
2,915691,30,0,2,0,0,청년,소,0,0,0,1,0,0,0,TRAIN
3,915940,28,0,1,1,0,청년,소,0,0,0,1,0,0,0,TEST
4,91594602,68,4,6,0,1,노년,소,1,1,0,1,1,0,0,TRAIN


In [45]:
D3_train_X = D3_2[D3_2['mode']=='TRAIN'][['Alkhol','familyhistory','고','소','노년','소년','중년']]
D3_train_y = D3_2[D3_2['mode']=='TRAIN'].Result
D3_test_X = D3_2[D3_2['mode']=='TEST'][['Alkhol','familyhistory','고','소','노년','소년','중년']]
D3_test_y = D3_2[D3_2['mode']=='TEST'].Result

In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
model = LogisticRegression(penalty='l2', random_state=13, solver='newton-cg')

In [48]:
model.fit(D3_train_X, D3_train_y)

LogisticRegression(random_state=13, solver='newton-cg')

In [49]:
model.coef_

array([[ 2.97358442, -1.35810121, -0.04278023,  0.96623277,  4.05800525,
        -1.25662071,  4.60608783]])

In [50]:
model.intercept_

array([-11.09280794])

In [51]:
predict = model.predict(D3_test_X.values)

In [52]:
predict, D3_test_y.values

(array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 

In [53]:
from sklearn.metrics import f1_score


In [54]:
f1_score(D3_test_y.values, predict)

1.0

In [55]:
proba = model.predict_proba(D3_test_X.values)

In [56]:
model.classes_

array([0, 1])

In [57]:
predict_from_proba = np.where(proba[:,1]>0.9,1,0)

In [58]:
f1_score(D3_test_y.values, predict_from_proba)

0.9565217391304348

In [59]:
'3번 문제 F-1 score는: {:.3f}'.format(f1_score(D3_test_y.values, predict_from_proba))

'3번 문제 F-1 score는: 0.957'

In [61]:
pd.DataFrame(data= {'predict': predict, 'predict_from_proba': predict_from_proba, 'true': D3_test_y.values}).values

array([[1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1,